# Corona Virus in the US

In [25]:
import pandas as pd
import numpy as np

### Aquire

In [79]:
df = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv")

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17731 entries, 0 to 17730
Data columns (total 6 columns):
date      17731 non-null object
county    17731 non-null object
state     17731 non-null object
fips      17452 non-null float64
cases     17731 non-null int64
deaths    17731 non-null int64
dtypes: float64(1), int64(2), object(3)
memory usage: 831.3+ KB


In [81]:
df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


### Prep

In [82]:
df.isnull().sum()

date        0
county      0
state       0
fips      279
cases       0
deaths      0
dtype: int64

In [83]:
df [df.fips.isnull()]

,date,county,state,fips,cases,deaths
416,2020-03-01,New York City,New York,NaN,1,0
418,2020-03-01,Unknown,Rhode Island,NaN,2,0
448,2020-03-02,New York City,New York,NaN,1,0
450,2020-03-02,Unknown,Rhode Island,NaN,2,0
482,2020-03-03,New York City,New York,NaN,2,0
...,...,...,...,...,...,...
17266,2020-03-28,Unknown,Rhode Island,NaN,168,2
17393,2020-03-28,Unknown,Tennessee,NaN,161,4
17523,2020-03-28,Unknown,Utah,NaN,17,1
17537,2020-03-28,Unknown,Vermont,NaN,23,5


In [84]:
df [df.fips.isnull()].county.value_counts()

Unknown          243
New York City     28
Kansas City        8
Name: county, dtype: int64

### Explore

In [121]:
states = df.groupby(df.state).sum()[["cases","deaths"]]

In [122]:
states["death_rate"] = states.deaths / states.cases * 100

In [123]:
states.index.name= None

In [124]:
states.sort_values("death_rate")

,cases,deaths,death_rate
Wyoming,471,0,0.000000
West Virginia,436,0,0.000000
Virgin Islands,147,0,0.000000
Northern Mariana Islands,2,0,0.000000
Hawaii,878,0,0.000000
Rhode Island,1432,2,0.139665
Maine,1295,2,0.154440
North Carolina,4629,10,0.216029
Utah,3145,8,0.254372
Montana,664,2,0.301205
